## With Tensorflow Keras

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import spacy

2024-08-03 09:56:22.013054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 09:56:22.013161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 09:56:22.149844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
start_char = 1
oov_char = 2
index_from = 3

In [3]:
(x_train, y_train), _ = tf.keras.datasets.imdb.load_data(start_char=start_char, oov_char=oov_char, index_from=index_from)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
x_train.shape

(25000,)

In [5]:
x_train.dtype

dtype('O')

In [6]:
y_train.shape

(25000,)

In [7]:
y_train[:5]

array([1, 0, 0, 1, 0])

In [8]:
df = pd.concat([pd.DataFrame({'X': x_train, 'Y': y_train})], ignore_index=True)
df.head()

,X,Y
0,"[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...",1
1,"[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...",0
2,"[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...",0
3,"[1, 4, 18609, 16085, 33, 2804, 4, 2040, 432, 1...",1
4,"[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...",0


In [9]:
word_index = tf.keras.datasets.imdb.get_word_index()

inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)

inverted_word_index[start_char] = ""
inverted_word_index[oov_char] = ""


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [10]:
print(len(word_index))
print(len(inverted_word_index))

88584
88586


In [11]:
encode = lambda text : [word_index[word] for word in text.split()]
decode = lambda inp : " ".join([inverted_word_index[idx] for idx in inp])

In [12]:
df["text"] = df["X"].apply(decode)

In [13]:
nlp = spacy.load("en_core_web_sm")
def remove_stop_words(text):
    doc = nlp(text)
    filtered = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered.append(token.lemma_)
    return " ".join(filtered)

In [14]:
df["pre_text"] = df["text"].apply(remove_stop_words)

In [15]:
nlp = spacy.load("en_core_web_lg")
df["vector"] = df["pre_text"].apply(lambda text: nlp(text).vector)

In [16]:
df.head()

,X,Y,text,pre_text,vector
0,"[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...",1,this film was just brilliant casting location...,film brilliant cast location scenery story d...,"[-0.49241343, 0.22992784, -1.9614998, -2.44784..."
1,"[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...",0,big hair big boobs bad music and a giant safe...,big hair big boob bad music giant safety pin...,"[-0.4520517, 0.6301006, -1.7863605, -1.3567178..."
2,"[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...",0,this has to be one of the worst films of the ...,bad film 1990 friend watch film target audie...,"[-0.58676946, 1.1265762, -2.6766188, -1.314009..."
3,"[1, 4, 18609, 16085, 33, 2804, 4, 2040, 432, 1...",1,the scots excel at storytelling the tradition...,scot excel storytelle traditional sort year ...,"[-0.12821278, 0.26851207, -1.1525749, -1.29237..."
4,"[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...",0,worst mistake of my life br br i picked this ...,bad mistake life br br pick movie target 5 f...,"[-0.37964636, 0.45462498, -2.716358, -0.844779..."


In [17]:
scaler = MinMaxScaler()

In [18]:
X = df["vector"].to_numpy()
X = np.array(X.tolist())
X.shape

(25000, 300)

In [19]:
X = scaler.fit_transform(X)

In [20]:
Y = df["Y"].to_numpy().astype(np.uint8)

In [21]:
X_train, X_test, y_train, y_test =  train_test_split(X, Y, test_size=0.2, random_state=42)

In [22]:
X_train.shape

(20000, 300)

In [23]:
y_train.shape

(20000,)

In [24]:
input_size = 300
num_classes = 2
batch_size = 16
dropout = 0.2

In [25]:
model = Sequential([
    Dense(input_size, activation="relu"),
    Dropout(dropout),
    BatchNormalization(),
    
    Dense(300, activation="relu"),
    Dropout(dropout),
    BatchNormalization(),
    
    Dense(100, activation="relu"),
    Dropout(dropout),
    BatchNormalization(),
    
    Dense(100, activation="relu"),
    Dropout(dropout),
    BatchNormalization(),
    
    Dense(num_classes, activation="softmax")
])

In [26]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [27]:
X_train = tf.cast(X_train, dtype=tf.float32)  
y_train = tf.cast(y_train, dtype=tf.int32)  
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)

Epoch 1/20
 73/500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5060 - loss: 1.0538

I0000 00:00:1722680577.628401      67 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.5481 - loss: 0.8653 - val_accuracy: 0.5020 - val_loss: 0.7375
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6465 - loss: 0.6281 - val_accuracy: 0.6267 - val_loss: 0.6136
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7155 - loss: 0.5525 - val_accuracy: 0.7190 - val_loss: 0.5487
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7193 - loss: 0.5246 - val_accuracy: 0.7745 - val_loss: 0.4671
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7361 - loss: 0.5010 - val_accuracy: 0.5153 - val_loss: 1.2500
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7488 - loss: 0.4838 - val_accuracy: 0.5115 - val_loss: 1.5112
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7500 - loss: 0.4824 - val_accuracy: 0.6348 - val_loss: 0.7724
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7492 - loss: 0.4781 - val_accuracy: 0.6043 - val